In [24]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings(action='ignore')

In [25]:
train_data = pd.read_csv("train.csv")
test_data = pd.read_csv("test.csv")
submission = pd.read_csv("sample_submission.csv")
train_data["log_target"] = np.log1p(train_data["target"])

all_data = pd.concat([train_data, test_data], ignore_index=True)
train_data.shape, test_data.shape, submission.shape

((1015, 12), (436, 10), (436, 2))

In [26]:
all_data["com"] = all_data["title"].map(lambda x: x.split()[0].upper())
all_data["com"].replace("MERCEDES-BENZ/52","MERCEDES-BENZ",inplace=True)

In [27]:
all_data["title"] = all_data["title"].apply(lambda x: x.upper())

In [28]:
all_data["new"] = all_data["odometer"].copy()
all_data["new"][all_data["new"]!=0] = 1
all_data["new"][all_data["new"]==0] = 0
all_data["log_odometer"] = all_data["odometer"].copy()
all_data["log_odometer"][all_data["log_odometer"]==0]=10
all_data["log_odometer"] = np.log(all_data["log_odometer"])
all_data["too_old"] = all_data["log_odometer"].copy()
all_data["too_old"][all_data["too_old"]<10] = 0
all_data["too_old"][all_data["too_old"]>=10] = 1

In [29]:
# year - 2020
all_data["old"] = all_data["year"]-2020
# 0 보다 큰 값은 0으로 수정
all_data["old"][all_data["old"]>0] = 0
# old 의 median = -10
all_data["old"][all_data["old"]<-100] = -10
# old 의  abs
all_data["old"] = abs(all_data["old"])

In [30]:
import re 

def clean_text(texts): 
    corpus = [] 
    for i in range(0, len(texts)): 
        review = re.sub(r'[@%\\*=()/~#&\+á?\xc3\xa1\-\|\.\:\;\!\-\,\_\~\$\'\"\n\]\[\>\<]', '',texts[i]) #@%*=()/+ 와 같은 문장부호 제거
        review = re.sub(r'\d+','',review)#숫자 제거
        review = review.lower() #소문자 변환
        review = re.sub(r'\s+', ' ', review) #extra space 제거
        review = re.sub(r'<[^>]+>','',review) #Html tags 제거
        review = re.sub(r'\s+', ' ', review) #spaces 제거
        review = re.sub(r"^\s+", '', review) #space from start 제거
        review = re.sub(r'\s+$', '', review) #space from the end 제거
        review = re.sub(r'_', ' ', review) #space from the end 제거
        #review = re.sub(r'l', '', review)
        corpus.append(review) 
        
    return corpus

temp = clean_text(all_data['paint']) #메소드 적용
all_data['paint'] = temp

In [31]:
all_data['paint'] = all_data['paint'] = all_data['paint'].apply(lambda x : 'blue' if x.find('blue') >= 0 else x)
all_data['paint'] = all_data['paint'] = all_data['paint'].apply(lambda x : 'red' if x.find('red') >= 0 else x)
all_data['paint'] = all_data['paint'] = all_data['paint'].apply(lambda x : 'green' if x.find('green') >= 0 else x)
all_data['paint'] = all_data['paint'] = all_data['paint'].apply(lambda x : 'white' if x.find('white') >= 0 else x)
all_data['paint'] = all_data['paint'] = all_data['paint'].apply(lambda x : 'grey' if x.find('grey') >= 0 else x)
all_data['paint'] = all_data['paint'] = all_data['paint'].apply(lambda x : 'grey' if x.find('gery') >= 0 else x)
all_data['paint'] = all_data['paint'] = all_data['paint'].apply(lambda x : 'grey' if x.find('gray') >= 0 else x)
all_data['paint'] = all_data['paint'] = all_data['paint'].apply(lambda x : 'ash' if x.find('ash') >= 0 else x)
all_data['paint'] = all_data['paint'] = all_data['paint'].apply(lambda x : 'brown' if x.find('brown') >= 0 else x)
all_data['paint'] = all_data['paint'] = all_data['paint'].apply(lambda x : 'silver' if x.find('silver') >= 0 else x)
all_data['paint'] = all_data['paint'] = all_data['paint'].apply(lambda x : 'silver' if x.find('sliver') >= 0 else x)
all_data['paint'] = all_data['paint'] = all_data['paint'].apply(lambda x : 'black' if x.find('black') >= 0 else x)
all_data['paint'] = all_data['paint'] = all_data['paint'].apply(lambda x : 'gold' if x.find('gold') >= 0 else x)
all_data['paint'] = all_data['paint'] = all_data['paint'].apply(lambda x : 'wine' if x.find('whine') >= 0 else x)

In [32]:
def replacestringetc(data,x, counts):
    if sum(data == x) >counts:
        return x
    else:
        return "etc"
    
all_data["paint"] = all_data["paint"].map(lambda x: replacestringetc(all_data["paint"], x, 3))

In [33]:
all_data["engine_type"] = all_data["engine"].copy()
all_data["cylinder"] = all_data["engine"].copy()
all_data["cylinder"] = all_data["cylinder"].map(lambda x: x[-2])
all_data["engine_type"] = all_data["engine_type"].map(lambda x: x[-3])

In [34]:
all_data["location"] = all_data["location"].map(lambda x: replacestringetc(all_data["location"],x,1))

In [35]:
all_data.head(3)

,id,title,odometer,location,isimported,engine,transmission,fuel,paint,year,target,log_target,com,new,log_odometer,too_old,old,engine_type,cylinder
0,0,TOYOTA RAV 4,18277,Lagos,Foreign Used,4-cylinder(I4),automatic,petrol,red,2016,13665000.0,16.430348,TOYOTA,1,9.813399,0.0,4,I,4
1,1,TOYOTA LAND CRUISER,10,Lagos,New,4-cylinder(I4),automatic,petrol,black,2019,33015000.0,17.312473,TOYOTA,1,2.302585,0.0,1,I,4
2,2,LAND ROVER RANGE ROVER EVOQUE,83091,Lagos,Foreign Used,6-cylinder(V6),automatic,petrol,red,2012,9915000.0,16.109559,LAND,1,11.327692,1.0,8,V,6


In [36]:
drop_feature = ["id","year","target","log_odometer","engine"]
all_data = all_data.drop(drop_feature, axis=1)
all_data["too_old"] = all_data["too_old"].astype(int)

In [37]:
all_data

,title,odometer,location,isimported,transmission,fuel,paint,log_target,com,new,too_old,old,engine_type,cylinder
0,TOYOTA RAV 4,18277,Lagos,Foreign Used,automatic,petrol,red,16.430348,TOYOTA,1,0,4,I,4
1,TOYOTA LAND CRUISER,10,Lagos,New,automatic,petrol,black,17.312473,TOYOTA,1,0,1,I,4
2,LAND ROVER RANGE ROVER EVOQUE,83091,Lagos,Foreign Used,automatic,petrol,red,16.109559,LAND,1,1,8,V,6
3,LEXUS ES 350,91524,Lagos,Foreign Used,automatic,petrol,grey,15.154451,LEXUS,1,1,13,I,4
4,TOYOTA VENZA,94177,Lagos,Foreign Used,automatic,petrol,red,15.814962,TOYOTA,1,1,10,V,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1446,MERCEDES-BENZ GLK 350,78175,Lagos,Foreign Used,automatic,petrol,blue,NaN,MERCEDES-BENZ,1,1,6,V,6
1447,HONDA CROSSTOUR,129223,Lagos,Foreign Used,automatic,petrol,red,NaN,HONDA,1,1,9,V,6
1448,MERCEDES-BENZ ML 350,100943,Lagos,Foreign Used,automatic,petrol,black,NaN,MERCEDES-BENZ,1,1,7,I,4
1449,LEXUS GX 470,81463,Lagos,Foreign Used,automatic,petrol,green,NaN,LEXUS,1,1,17,I,4


In [15]:
temp = pd.get_dummies(all_data[["title","location","isimported","transmission","fuel","paint","com","engine_type"]])
temp = pd.concat([all_data,temp],axis=1)

In [16]:
all_data = temp.drop(["title","location","isimported","transmission","fuel","paint","com","engine_type"], axis=1)

In [17]:
from sklearn.preprocessing import MinMaxScaler
X_scaler = MinMaxScaler()
y_scaler = MinMaxScaler()

In [18]:
X = all_data.drop("log_target", axis=1)
y = all_data[["log_target"]]
len(all_data[~all_data["log_target"].isnull()])

1015

In [19]:
X = X_scaler.fit_transform(X)
y = y_scaler.fit_transform(y)

In [20]:
X_train = X[:1015]
X_test = X[1015:]
y_train = y[:1015]

In [21]:
X_train.shape,X_test.shape, y_train.shape

((1015, 322), (436, 322), (1015, 1))

In [53]:
# 평가산식
from sklearn.metrics import make_scorer

def nmae(answer, pred):
    mae = np.mean(np.abs(answer-pred))
    score = mae / np.mean(np.abs(answer))
    return score

NMAE = make_scorer(nmae, greater_is_better=False)

def at_NMAE(y_pred, dataset):
    y_true = dataset.get_label()
    
    mae = np.mean(np.abs(y_true-y_pred))
    score = mae / np.mean(np.abs(y_true))

    return "score", score, False

# 모델링

## XGB

In [37]:
# XGB
pbounds = { 'learning_rate': (0.0001, 0.4),
            'n_estimators': (16, 1024),
            'max_depth': (3,10),   
            'subsample': (0.5, 1),  
            'colsample_bytree': (0.5, 1),   
            'gamma': (0, 5)}

def xgb_opt(learning_rate, n_estimators, max_depth, subsample, colsample_bytree, gamma):

    params = {
        'learning_rate': learning_rate,
        'n_estimators' : int(round(n_estimators)),
        'max_depth' : int(round(max_depth)),
        'subsample': subsample,
        'colsample': colsample_bytree,   
        'gamma': gamma,
        'n_jobs' : -1,
        'random_state' : 777
    }
    
    xgb = XGBRegressor(**params)
    
    kf = KFold(n_splits = 4 , shuffle=True, random_state=50)

    score = cross_val_score(xgb, X_train, y_train, scoring=NMAE, cv=kf, n_jobs=-1)
    
    return np.mean(score)

BO_xgb = BayesianOptimization(f = xgb_opt, pbounds = pbounds, random_state=0)

In [38]:
BO_xgb.maximize(init_points = 10, n_iter = 200)

|   iter    |  target   | colsam... |   gamma   | learni... | max_depth | n_esti... | subsample |
-------------------------------------------------------------------------------------------------
|  1        | -0.3221   |  0.7744   |  3.576    |  0.2411   |  6.814    |  443.0    |  0.8229   |
|  2        | -0.3185   |  0.7188   |  4.459    |  0.3855   |  5.684    |  814.1    |  0.7644   |
|  3        | -0.3247   |  0.784    |  4.628    |  0.02851  |  3.61     |  36.38    |  0.9163   |
|  4        | -0.3188   |  0.8891   |  4.35     |  0.3914   |  8.594    |  481.2    |  0.8903   |
|  5        | -0.3204   |  0.5591   |  3.2      |  0.05743  |  9.613    |  542.0    |  0.7073   |
|  6        | -0.3209   |  0.6323   |  3.871    |  0.1825   |  6.979    |  34.94    |  0.8088   |
|  7        | -0.3316   |  0.806    |  3.085    |  0.3775   |  7.773    |  378.4    |  0.7185   |
|  8        | -0.3815   |  0.8488   |  0.3011   |  0.2667   |  7.694    |  228.1    |  0.5645   |
|  9        | -0.338

|  51       | -0.3554   |  1.0      |  2.902    |  0.0001   |  3.0      |  537.7    |  0.5      |
|  52       | -0.3498   |  0.9336   |  1.192    |  0.2195   |  3.438    |  457.1    |  0.9566   |
|  53       | -0.3555   |  0.8789   |  5.0      |  0.0001   |  10.0     |  524.3    |  1.0      |
|  54       | -0.3178   |  0.803    |  4.538    |  0.1475   |  7.9      |  535.5    |  0.8872   |
|  55       | -0.3166   |  0.8125   |  4.256    |  0.2818   |  9.577    |  48.43    |  0.7077   |
|  56       | -0.3146   |  0.7569   |  4.6      |  0.03438  |  6.037    |  49.38    |  0.5448   |
|  57       | -0.3183   |  0.8882   |  4.685    |  0.3811   |  8.353    |  52.22    |  0.7239   |
|  58       | -0.3448   |  0.7715   |  0.6537   |  0.005513 |  7.424    |  50.93    |  0.7054   |
|  59       | -0.3193   |  0.6638   |  5.0      |  0.4      |  3.889    |  52.68    |  1.0      |
|  60       | -0.3193   |  0.5      |  5.0      |  0.4      |  3.0      |  46.72    |  1.0      |
|  61       | -0.316

|  102      | -0.316    |  0.9449   |  4.643    |  0.1789   |  8.258    |  447.6    |  0.622    |
|  103      | -0.3175   |  0.8873   |  4.602    |  0.208    |  4.711    |  481.8    |  0.8884   |
|  104      | -0.3138   |  0.5559   |  4.879    |  0.05133  |  5.616    |  806.7    |  0.8023   |
|  105      | -0.3448   |  0.5369   |  4.808    |  0.00312  |  9.718    |  72.14    |  0.5803   |
|  106      | -0.3163   |  0.6368   |  4.88     |  0.3351   |  6.365    |  808.6    |  0.9746   |
|  107      | -0.3165   |  0.6064   |  3.735    |  0.2755   |  3.833    |  72.08    |  0.5795   |
|  108      | -0.3185   |  0.8776   |  4.816    |  0.3804   |  3.928    |  74.41    |  0.8624   |
|  109      | -0.3193   |  0.5      |  5.0      |  0.4      |  10.0     |  40.31    |  1.0      |
|  110      | -0.3813   |  0.9807   |  0.4256   |  0.3817   |  5.877    |  71.24    |  0.8239   |
|  111      | -0.3534   |  1.0      |  5.0      |  0.0001   |  3.0      |  806.3    |  0.5      |
|  112      | -0.358

|  153      | -0.3154   |  0.5838   |  4.757    |  0.06685  |  6.73     |  41.88    |  0.8551   |
|  154      | -0.3205   |  0.5      |  5.0      |  0.4      |  3.435    |  449.7    |  0.5      |
|  155      | -0.3479   |  1.0      |  1.774    |  0.4      |  10.0     |  463.4    |  0.5      |
|  156      | -0.3164   |  0.5138   |  4.858    |  0.133    |  5.324    |  542.1    |  0.928    |
|  157      | -0.3199   |  0.8595   |  3.676    |  0.2435   |  3.535    |  76.91    |  0.7339   |
|  158      | -0.3196   |  1.0      |  5.0      |  0.4      |  8.836    |  37.36    |  0.5      |
|  159      | -0.3204   |  1.0      |  5.0      |  0.4      |  3.0      |  70.45    |  0.5      |
|  160      | -0.3162   |  0.8748   |  4.609    |  0.1133   |  3.461    |  90.11    |  0.9021   |
|  161      | -0.3242   |  0.9899   |  2.512    |  0.2676   |  3.531    |  92.85    |  0.5538   |
|  162      | -0.3145   |  0.9336   |  4.911    |  0.2747   |  5.863    |  92.2     |  0.5379   |
|  163      | -0.313

|  204      | -0.3115   |  0.5092   |  4.705    |  0.02053  |  9.081    |  774.9    |  0.5679   |
|  205      | -0.3236   |  1.0      |  2.403    |  0.1888   |  8.037    |  775.3    |  0.5      |
|  206      | -0.3538   |  0.5      |  5.0      |  0.0001   |  7.24     |  773.0    |  1.0      |
|  207      | -0.3205   |  1.0      |  5.0      |  0.4      |  9.835    |  443.3    |  0.5      |
|  208      | -0.3559   |  0.5      |  5.0      |  0.0001   |  3.0      |  460.0    |  1.0      |
|  209      | -0.3161   |  0.7569   |  4.836    |  0.1273   |  9.129    |  780.8    |  0.7623   |
|  210      | -0.3193   |  1.0      |  5.0      |  0.4      |  7.328    |  457.7    |  1.0      |


In [44]:
BO_xgb.max

{'target': -0.3114613401742714,
 'params': {'colsample_bytree': 0.6086311591938874,
  'gamma': 4.4823554111740975,
  'learning_rate': 0.06756917692030592,
  'max_depth': 8.951621721309039,
  'n_estimators': 90.6237357490009,
  'subsample': 0.5084855077624653}}

In [45]:
params = BO_xgb.max['params']
params['max_depth'] = int(round(params['max_depth']))
params['n_estimators'] = int(round(params['n_estimators']))

In [46]:
xgb = XGBRegressor(random_state = 42, **params)

In [47]:
xgb.fit(X_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.6086311591938874,
             gamma=4.4823554111740975, gpu_id=-1, importance_type='gain',
             interaction_constraints='', learning_rate=0.06756917692030592,
             max_delta_step=0, max_depth=9, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=91, n_jobs=16,
             num_parallel_tree=1, random_state=42, reg_alpha=0, reg_lambda=1,
             scale_pos_weight=1, subsample=0.5084855077624653,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [109]:
y_pred_xgb = y_scaler.inverse_transform([xgb.predict(X_test)])
y_pred_xgb = np.expm1(y_pred)

## 엑스트라트리

In [59]:
from sklearn.ensemble import ExtraTreesRegressor

In [68]:
param = {'max_depth' : [2,3,4,5,6,7,8,9,10], 'n_estimators' : [100,150,200,400,600,1000]}
grid_et = GridSearchCV(ExtraTreesRegressor(random_state = 42), param, cv = KFold(n_splits = 5, shuffle = True, random_state = 1011), scoring = NMAE, n_jobs = -1)

In [70]:
grid_et.fit(X_train, y_train)

GridSearchCV(cv=KFold(n_splits=5, random_state=1011, shuffle=True),
             estimator=ExtraTreesRegressor(random_state=42), n_jobs=-1,
             param_grid={'max_depth': [2, 3, 4, 5, 6, 7, 8, 9, 10],
                         'n_estimators': [100, 150, 200, 400, 600, 1000]},
             scoring=make_scorer(nmae, greater_is_better=False))

In [71]:
et = ExtraTreesRegressor(random_state = 42, **grid_et.best_params_)

In [72]:
et.fit(X_train, y_train)

ExtraTreesRegressor(max_depth=2, n_estimators=1000, random_state=42)

In [76]:
y_pred = y_scaler.inverse_transform([et.predict(X_test)])
y_pred_et = np.expm1(y_pred)

## 베이지안 릿지

In [78]:
from sklearn.linear_model import BayesianRidge, LinearRegression

In [81]:
 br = BayesianRidge(alpha_1=1e-06, alpha_2=1e-06, alpha_init=None,
               compute_score=False, copy_X=True, fit_intercept=True,
               lambda_1=1e-06, lambda_2=1e-06, lambda_init=None, n_iter=300,
               normalize=False, tol=0.001, verbose=False)

In [82]:
br.fit(X_train, y_train)

BayesianRidge()

In [85]:
y_pred = y_scaler.inverse_transform([br.predict(X_test)])
y_pred_br = np.expm1(y_pred)

## 그라디언트부스팅

In [86]:
pbounds = { 'learning_rate': (0.0005, 0.2),
            'n_estimators': (16, 512),
            'max_depth': (3,10),   
            'subsample': (0.5,1), 
            'min_samples_split': (2,5),   
            'min_samples_leaf': (1,5)}

def gbm_opt(learning_rate, n_estimators, max_depth, subsample, min_samples_split, min_samples_leaf):

    params = {
        'learning_rate': learning_rate,
        'n_estimators' : int(round(n_estimators)),
        'max_depth' : int(round(max_depth)),
        'subsample': subsample,
        'min_samples_split' : int(round(min_samples_split)),
        'min_samples_leaf' : int(round(min_samples_leaf))
    }
    
    gbm = GradientBoostingRegressor(**params)
    kf = KFold(n_splits = 4 , shuffle = True, random_state=50)
    score = cross_val_score(gbm, X_train, y_train, scoring=NMAE, cv = kf, n_jobs=-1)
    return np.mean(score)

BO_gbm = BayesianOptimization(f = gbm_opt, pbounds = pbounds, random_state=0)    

In [87]:
BO_gbm.maximize(init_points = 10, n_iter = 200)

|   iter    |  target   | learni... | max_depth | min_sa... | min_sa... | n_esti... | subsample |
-------------------------------------------------------------------------------------------------
|  1        | -0.4226   |  0.11     |  8.006    |  3.411    |  3.635    |  226.1    |  0.8229   |
|  2        | -0.4252   |  0.0878   |  9.242    |  4.855    |  3.15     |  408.7    |  0.7644   |
|  3        | -0.4104   |  0.1138   |  9.479    |  1.284    |  2.261    |  26.03    |  0.9163   |
|  4        | -0.4247   |  0.1557   |  9.09     |  4.914    |  4.397    |  244.9    |  0.8903   |
|  5        | -0.419    |  0.0241   |  7.479    |  1.573    |  4.834    |  274.8    |  0.7073   |
|  6        | -0.3715   |  0.05328  |  8.42     |  2.825    |  3.705    |  25.32    |  0.8088   |
|  7        | -0.4243   |  0.1226   |  7.319    |  4.775    |  4.045    |  194.3    |  0.7185   |
|  8        | -0.4226   |  0.1397   |  3.422    |  3.667    |  4.012    |  120.3    |  0.5645   |
|  9        | -0.425

|  51       | -0.3969   |  0.1226   |  4.429    |  4.304    |  2.922    |  22.48    |  0.7288   |
|  52       | -0.3849   |  0.08172  |  7.716    |  4.014    |  4.193    |  20.57    |  0.5701   |
|  53       | -0.4092   |  0.1516   |  7.867    |  3.593    |  4.525    |  19.4     |  0.8858   |
|  54       | -0.4169   |  0.01301  |  6.911    |  1.494    |  4.334    |  424.9    |  0.8195   |
|  55       | -0.4228   |  0.09413  |  3.645    |  2.767    |  4.872    |  276.8    |  0.7509   |
|  56       | -0.3616   |  0.05492  |  4.706    |  3.034    |  2.458    |  22.12    |  0.8164   |
|  57       | -0.4243   |  0.09049  |  8.35     |  4.667    |  3.855    |  252.6    |  0.6121   |
|  58       | -0.4256   |  0.06054  |  8.092    |  4.586    |  2.942    |  370.0    |  0.6423   |
|  59       | -0.3334   |  0.02945  |  4.782    |  2.911    |  2.521    |  22.09    |  0.7085   |
|  60       | -0.4241   |  0.09861  |  9.048    |  2.786    |  3.764    |  322.2    |  0.8459   |
|  61       | -0.426

|  102      | -0.4249   |  0.1851   |  8.591    |  1.661    |  3.514    |  214.3    |  0.8894   |
|  103      | -0.3175   |  0.01259  |  8.273    |  3.124    |  3.463    |  24.67    |  0.7751   |
|  104      | -0.4249   |  0.09092  |  4.835    |  1.41     |  3.299    |  460.2    |  0.721    |
|  105      | -0.4242   |  0.09987  |  8.889    |  3.246    |  3.033    |  220.0    |  0.5792   |
|  106      | -0.4165   |  0.05002  |  9.245    |  4.8      |  4.116    |  85.37    |  0.6827   |
|  107      | -0.4221   |  0.04335  |  3.241    |  1.626    |  4.589    |  471.1    |  0.5544   |
|  108      | -0.3862   |  0.08685  |  6.834    |  3.137    |  4.752    |  20.06    |  0.9288   |
|  109      | -0.3821   |  0.0811   |  5.099    |  3.277    |  2.12     |  22.27    |  0.5962   |
|  110      | -0.4229   |  0.1405   |  4.24     |  3.973    |  3.565    |  156.3    |  0.6633   |
|  111      | -0.4234   |  0.1545   |  8.848    |  1.06     |  2.371    |  478.7    |  0.503    |
|  112      | -0.396

|  153      | -0.3354   |  0.03276  |  8.216    |  3.805    |  3.99     |  20.13    |  0.5454   |
|  154      | -0.4213   |  0.03777  |  7.792    |  2.865    |  2.391    |  228.7    |  0.5666   |
|  155      | -0.4045   |  0.1194   |  8.299    |  4.148    |  3.786    |  20.95    |  0.5393   |
|  156      | -0.4153   |  0.05222  |  6.612    |  4.021    |  2.393    |  84.69    |  0.795    |
|  157      | -0.4103   |  0.1978   |  4.086    |  3.283    |  3.214    |  23.15    |  0.5623   |
|  158      | -0.3698   |  0.05356  |  8.274    |  3.082    |  3.508    |  24.34    |  0.738    |
|  159      | -0.347    |  0.01005  |  4.092    |  2.19     |  3.28     |  95.73    |  0.7393   |
|  160      | -0.4254   |  0.1946   |  9.66     |  2.302    |  4.922    |  401.8    |  0.6102   |
|  161      | -0.3096   |  0.0005   |  8.021    |  3.641    |  4.536    |  19.92    |  0.634    |
|  162      | -0.4004   |  0.01294  |  9.617    |  2.835    |  3.02     |  180.1    |  0.8488   |
|  163      | -0.329

|  204      | -0.3097   |  0.001985 |  7.805    |  4.054    |  3.73     |  20.3     |  0.5307   |
|  205      | -0.4153   |  0.1885   |  7.58     |  3.799    |  3.654    |  19.72    |  0.9889   |
|  206      | -0.4136   |  0.1653   |  8.291    |  3.644    |  4.333    |  21.0     |  0.7161   |
|  207      | -0.4      |  0.05421  |  7.452    |  4.281    |  4.37     |  45.22    |  0.7141   |
|  208      | -0.4069   |  0.1328   |  7.874    |  3.226    |  4.883    |  19.65    |  0.7091   |
|  209      | -0.4248   |  0.07622  |  4.666    |  1.562    |  2.458    |  413.3    |  0.8491   |
|  210      | -0.4237   |  0.07174  |  7.774    |  1.689    |  2.786    |  456.5    |  0.7523   |


In [88]:
BO_gbm.max

{'target': -0.3096418485805604,
 'params': {'learning_rate': 0.0005,
  'max_depth': 8.021138569018108,
  'min_samples_leaf': 3.6406549269759805,
  'min_samples_split': 4.53609633372976,
  'n_estimators': 19.918557538720293,
  'subsample': 0.6339769535524644}}

In [89]:
params = BO_gbm.max['params']
params['max_depth'] = int(round(params['max_depth']))
params['n_estimators'] = int(round(params['n_estimators']))
params['min_samples_leaf'] = int(round(params['min_samples_leaf']))
params['min_samples_split'] = int(round(params['min_samples_split']))

In [90]:
gbm = GradientBoostingRegressor(random_state = 42, **params)
gbm.fit(X_train, y_train)

GradientBoostingRegressor(learning_rate=0.0005, max_depth=8, min_samples_leaf=4,
                          min_samples_split=5, n_estimators=20, random_state=42,
                          subsample=0.6339769535524644)

In [94]:
y_pred = y_scaler.inverse_transform([gbm.predict(X_test)])
y_pred_gbm = np.expm1(y_pred)

## votingRegressor

In [147]:
vot = VotingRegressor(estimators = [('br', br), ('xgb' ,xgb), ('gbr', gbr), ('et', et)], weights = [.2, .4, .1, .3])

In [148]:
vot.fit(X_train, y_train)

VotingRegressor(estimators=[('br', BayesianRidge()),
                            ('xgb',
                             XGBRegressor(base_score=0.5, booster='gbtree',
                                          colsample_bylevel=1,
                                          colsample_bynode=1,
                                          colsample_bytree=0.6086311591938874,
                                          gamma=4.4823554111740975, gpu_id=-1,
                                          importance_type='gain',
                                          interaction_constraints='',
                                          learning_rate=0.06756917692030592,
                                          max_delta_step=0, max_depth=9,
                                          min_child_weight=1, missi...
                                          monotone_constraints='()',
                                          n_estimators=91, n_jobs=16,
                                          num_parallel_t

In [149]:
y_pred = y_scaler.inverse_transform([vot.predict(X_test)])
y_pred_vot = np.expm1(y_pred)

## 앙상블

In [105]:
submission

,id,target
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0
...,...,...
431,431,0
432,432,0
433,433,0
434,434,0


In [166]:
temp = (y_pred_xgb*y_pred_br*y_pred_et*y_pred_gbm*y_pred_vot) ** (1/5)

In [167]:
temp = temp.reshape(-1)
submission['target'] = temp

In [168]:
submission.to_csv("submission_8.csv", index=False)
pd.read_csv("submission_8.csv")

,id,target
0,0,1.073861e+07
1,1,5.247044e+06
2,2,5.937264e+06
3,3,2.226318e+06
4,4,3.850711e+06
...,...,...
431,431,6.972575e+06
432,432,5.103097e+06
433,433,6.523353e+06
434,434,3.988343e+06


In [142]:
 gbr = GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0,
                           init=None, learning_rate=0.1, loss='ls', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None,
                           random_state=8293, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0, warm_start=False)

In [143]:
gbr.fit(X_train,y_train)

GradientBoostingRegressor(random_state=8293)

In [145]:
y_pred = y_scaler.inverse_transform([gbr.predict(X_test)])
y_pred_gbr = np.expm1(y_pred)

In [47]:
X_train = all_data[~all_data["log_target"].isnull()]
X_test = all_data[all_data["log_target"].isnull()]
X_test = X_test.reset_index()
X_test = X_test.drop(["log_target","index"], axis=1)

In [48]:
X_train.head(3)

,title,odometer,location,isimported,transmission,fuel,paint,log_target,com,new,too_old,old,engine_type,cylinder
0,TOYOTA RAV 4,18277,Lagos,Foreign Used,automatic,petrol,red,16.430348,TOYOTA,1,0,4,I,4
1,TOYOTA LAND CRUISER,10,Lagos,New,automatic,petrol,black,17.312473,TOYOTA,1,0,1,I,4
2,LAND ROVER RANGE ROVER EVOQUE,83091,Lagos,Foreign Used,automatic,petrol,red,16.109559,LAND,1,1,8,V,6


In [49]:
X_test

,title,odometer,location,isimported,transmission,fuel,paint,com,new,too_old,old,engine_type,cylinder
0,MERCEDES-BENZ C 300,1234,Abuja,New,automatic,petrol,white,MERCEDES-BENZ,1,0,3,I,4
1,HONDA ACCORD,29938,Abuja,Foreign Used,automatic,petrol,white,HONDA,1,1,7,I,4
2,MERCEDES-BENZ S 550,87501,Lagos,Foreign Used,automatic,petrol,black,MERCEDES-BENZ,1,1,8,I,4
3,TOYOTA SIENNA,180894,Lagos,Locally used,automatic,petrol,grey,TOYOTA,1,1,19,V,6
4,TOYOTA HIACE,104814,Lagos,Foreign Used,automatic,petrol,white,TOYOTA,1,1,20,I,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
431,MERCEDES-BENZ GLK 350,78175,Lagos,Foreign Used,automatic,petrol,blue,MERCEDES-BENZ,1,1,6,V,6
432,HONDA CROSSTOUR,129223,Lagos,Foreign Used,automatic,petrol,red,HONDA,1,1,9,V,6
433,MERCEDES-BENZ ML 350,100943,Lagos,Foreign Used,automatic,petrol,black,MERCEDES-BENZ,1,1,7,I,4
434,LEXUS GX 470,81463,Lagos,Foreign Used,automatic,petrol,green,LEXUS,1,1,17,I,4


In [50]:
from pycaret.regression import *

In [51]:
data = setup(data=X_train, target="log_target", train_size=0.8)

,Description,Value
0,session_id,615
1,Target,log_target
2,Original Data,"(1015, 14)"
3,Missing Values,False
4,Numeric Features,3
5,Categorical Features,10
6,Ordinal Features,False
7,High Cardinality Features,False
8,High Cardinality Method,None
9,Transformed Train Set,"(812, 245)"


In [54]:
add_metric("nmae2","NMAE2",nmae, False)

Name                                                      NMAE2
Display Name                                              NMAE2
Score Function            <function nmae at 0x0000019231E25A60>
Scorer               make_scorer(nmae, greater_is_better=False)
Target                                                     pred
Args                                                         {}
Greater is Better                                         False
Custom                                                     True
Name: nmae2, dtype: object

In [55]:
compare_models(n_select=7, sort="NMAE2")

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,NMAE2,TT (Sec)
xgboost,Extreme Gradient Boosting,0.2382,0.1183,0.3422,0.8738,0.0207,0.0155,0.0155,0.2120
br,Bayesian Ridge,0.2487,0.1291,0.3571,0.8637,0.0217,0.0162,0.0162,0.0110
ridge,Ridge Regression,0.2510,0.1289,0.3568,0.8636,0.0217,0.0163,0.0163,0.0070
et,Extra Trees Regressor,0.2549,0.1457,0.3786,0.8457,0.0229,0.0165,0.0166,0.0750
gbr,Gradient Boosting Regressor,0.2609,0.1331,0.3627,0.8588,0.0219,0.0169,0.0170,0.0350
rf,Random Forest Regressor,0.2656,0.1531,0.3883,0.8387,0.0233,0.0171,0.0173,0.0590
lightgbm,Light Gradient Boosting Machine,0.3014,0.1771,0.4189,0.8136,0.0252,0.0195,0.0196,0.0130
omp,Orthogonal Matching Pursuit,0.3049,0.1906,0.4334,0.7957,0.0266,0.0198,0.0198,0.0060
dt,Decision Tree Regressor,0.3343,0.2311,0.4788,0.7547,0.0289,0.0216,0.0217,0.0060
ada,AdaBoost Regressor,0.3818,0.2455,0.4934,0.7429,0.0298,0.0248,0.0248,0.0380


[XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=-1, num_parallel_tree=1, objective='reg:squarederror',
              predictor='auto', random_state=615, reg_alpha=0, ...),
 BayesianRidge(alpha_1=1e-06, alpha_2=1e-06, alpha_init=None,
               compute_score=False, copy_X=True, fit_intercept=True,
               lambda_1=1e-06, lambda_2=1e-06, lambda_init=None, n_iter=300,
               normalize=False, tol=0.001, verbose=False),
 Ridge(alpha=1

In [56]:
blender = blend_models(estimator_list=compare_models(n_select=7, sort="NMAE2"))

,MAE,MSE,RMSE,R2,RMSLE,MAPE,NMAE2
Fold,,,,,,,
0,0.2136,0.0793,0.2815,0.9284,0.0171,0.0137,0.0137
1,0.2338,0.1623,0.4028,0.8310,0.0237,0.0148,0.0151
2,0.2213,0.0993,0.3151,0.8719,0.0194,0.0145,0.0145
3,0.2178,0.1070,0.3271,0.8988,0.0193,0.0140,0.0142
4,0.2224,0.1016,0.3188,0.9031,0.0196,0.0147,0.0146
5,0.2569,0.1620,0.4025,0.8358,0.0240,0.0167,0.0168
6,0.2263,0.1067,0.3266,0.8807,0.0196,0.0146,0.0147
7,0.2520,0.0967,0.3110,0.9078,0.0190,0.0164,0.0163
8,0.2305,0.0861,0.2934,0.9114,0.0180,0.0151,0.0150


In [57]:
final = finalize_model(blender)

In [59]:
y_pred = predict_model(final, data=X_test)

In [60]:
y_pred = y_pred.reset_index()

In [61]:
y_pred_exp = np.expm1(y_pred["Label"])

In [62]:
submission["target"] = y_pred_exp

In [63]:
submission.to_csv("submission_9.csv", index=False)
pd.read_csv("submission_9.csv")

,id,target
0,0,1.725737e+07
1,1,4.933374e+06
2,2,7.452176e+06
3,3,1.099029e+06
4,4,2.693722e+06
...,...,...
431,431,1.103373e+07
432,432,5.422465e+06
433,433,9.106959e+06
434,434,3.026632e+06
